In [20]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [21]:
util.startLoop()


In [22]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

Error 1100, reqId -1: Conectividad entre IB y TWS ha sido perdida
Peer closed connection.


In [23]:
# Definir el contrato para la acción de Palantir Technologies Inc. (PLTR)
contract = Stock('PLTR', 'SMART', 'USD')

In [24]:
# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=30)


In [25]:
# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='30 D',
    barSizeSetting='1 hour',
    whatToShow='TRADES',
    useRTH=False
)

In [26]:
# Convertir los datos a DataFrame
df = util.df(bars)


In [27]:
# Obtener noticias relevantes sobre Palantir Technologies Inc. (PLTR)
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
news_url = f'https://newsapi.org/v2/everything?q=Palantir%20OR%20NYSE%20PLTR&apiKey={API_KEY}'
response = requests.get(news_url)
news_data = response.json()


In [29]:
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")

Titulares de noticias relevantes:
 ['Palantir (PLTR) Shares Skyrocket, What You Need To Know', 'Palantir Sparks AI Frenzy with Teaser--Big Reveal on March 13', 'Palantir Technologies (PLTR) Stock Bulls Get Set to Punish Panicked Sellers', "Palantir won't be hurt by Pentagon cuts, analyst says — and the stock pops 5%", 'Palantir stock slips more than 8% as the bleeding gets worse', 'Nvidia, Tesla, Oracle, Palantir, Coinbase, Doordash: Stocks to watch today', 'Intel, Nvidia, Tesla, Palantir, IRobot: Stocks to watch today', 'Will Palantir Stock Fall To $40?', 'Walmart, Alibaba, Palantir, Meta, Hasbro, Block, Cruise Lines: Stocks to watch today', 'Shares in “Unique” Palantir ($PLTR) Rise On Hopes it Can Deflect Trump Cost Crunch', 'Morgan Stanley cautions that the S&P 500 could suffer a 5% drop', "Nvidia and Big Tech stocks can't escape the market's growing anxiety", '1 Artificial Intelligence (AI) Stock to Buy Hand Over Fist Before It Surges by 60%, According to 1 Wall Street Analyst', 'P

In [30]:
# Análisis de sentimiento de noticias
positive_keywords = ['rise', 'growth', 'record', 'profit']
negative_keywords = ['fall', 'loss', 'decline', 'drop']


In [31]:
positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))
impact_factor = 1 + (positive_news - negative_news) * 0.001

In [32]:
# Seleccionar datos y normalizar
data = df[['open', 'high', 'low', 'close']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
scaled_data_adjusted = scaled_data * impact_factor

In [35]:
# Identificar soportes y resistencias
def find_support_resistance(data):
    levels = []
    for i in range(2, len(data) - 2):
        if data[i] < data[i - 1] and data[i] < data[i + 1]:
            levels.append((i, data[i]))  # Soporte
        elif data[i] > data[i - 1] and data[i] > data[i + 1]:
            levels.append((i, data[i]))  # Resistencia
    return levels

support_resistance_levels = find_support_resistance(df['close'].values)
print(f"Niveles de soporte y resistencia detectados: {support_resistance_levels}")


Niveles de soporte y resistencia detectados: [(4, np.float64(110.92)), (8, np.float64(115.97)), (15, np.float64(110.29)), (18, np.float64(112.34)), (19, np.float64(111.86)), (20, np.float64(112.08)), (21, np.float64(111.75)), (25, np.float64(115.5)), (26, np.float64(115.33)), (28, np.float64(117.57)), (30, np.float64(117.05)), (31, np.float64(117.56)), (33, np.float64(116.3)), (35, np.float64(117.05)), (36, np.float64(115.4)), (37, np.float64(116.2)), (40, np.float64(113.39)), (41, np.float64(113.86)), (43, np.float64(112.64)), (44, np.float64(113.21)), (45, np.float64(113.14)), (46, np.float64(113.4)), (47, np.float64(113.25)), (49, np.float64(113.91)), (52, np.float64(110.36)), (54, np.float64(115.8)), (55, np.float64(115.62)), (60, np.float64(117.79)), (61, np.float64(117.67)), (62, np.float64(117.73)), (64, np.float64(116.73)), (65, np.float64(116.81)), (66, np.float64(116.51)), (68, np.float64(117.1)), (69, np.float64(116.08)), (70, np.float64(117.94)), (72, np.float64(116.28)), (

In [36]:
# Crear secuencias de datos para LSTM
sequence_length = 60
x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data_adjusted)):
    x_train.append(scaled_data_adjusted[i-sequence_length:i])
    y_train.append(scaled_data_adjusted[i])

x_train, y_train = np.array(x_train), np.array(y_train)

In [37]:
 # Definir modelo LSTM
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=4))
  
  # Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20, batch_size=32)
  

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.1761
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0227
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0146
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0087
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0088
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0080
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0077
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0077
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0069
Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0075
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0072
Epoch 12/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0078
Epoch 13/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0070
Epoch 14/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0072
Epoch 15/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0063
Epoc

In [38]:
# Predicciones
recent_data = scaled_data_adjusted[-sequence_length:]
predictions = []
for _ in range(10):
      x_input = np.array([recent_data])
      predicted_price = model.predict(x_input)
      predictions.append(predicted_price[0])
      recent_data = np.append(recent_data[1:], [predicted_price[0]], axis=0)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [39]:
predictions = scaler.inverse_transform(predictions)
prediction_times = [end_time.replace(hour=7, minute=30, second=0) + timedelta(hours=i) for i in range(1, 11)]
predicted_df = pd.DataFrame(predictions, columns=['open', 'high', 'low', 'close'], index=prediction_times)
print(predicted_df)

                                 open       high        low      close
2025-03-21 08:30:00.317733  86.041661  87.270247  84.971716  86.396835
2025-03-21 09:30:00.317733  85.966764  87.199811  84.908010  86.328274
2025-03-21 10:30:00.317733  85.893519  87.129053  84.845177  86.255093
2025-03-21 11:30:00.317733  85.819732  87.056505  84.781673  86.178669
2025-03-21 12:30:00.317733  85.744645  86.981899  84.717075  86.100284
2025-03-21 13:30:00.317733  85.668345  86.905630  84.651508  86.021028
2025-03-21 14:30:00.317733  85.591345  86.828395  84.585348  85.941775
2025-03-21 15:30:00.317733  85.514317  86.750945  84.519017  85.863217
2025-03-21 16:30:00.317733  85.437912  86.673985  84.452910  85.785864
2025-03-21 17:30:00.317733  85.362704  86.598109  84.387361  85.710087


In [19]:
ib.disconnect()